#### 랭체인(langchain) + PDF 기반 질의응답(Question-Answering)
<목차>
- langchain 의 문서 로드
- 분할
- 벡터스토어(vectorstore)에 임베딩된 문서를 저장 하는 방법

-----

#### Retrieval Augmented Generation(RAG)란?
- 파운데이션 모델을 특정 도메인에 대해 파인 튜닝(Fine-tuning)할 경우 모델의 성능을 크게 개선시킬 수 있지만 데이터 수집부터 학습에 사용해야 하는 클라우드 서버 대여까지 큰 비용이 발생
- 일례로 BloombergGPT의 경우 약 700B개(약 7000억개)의 토큰을 학습에 사용하였고,
- LLaMA2가 2조개의 토큰을 학습에 사용
- 보통 일반적인 PDF문서가 1000개의 토큰을 가지고 있다고 가정하면, 특정 도메인의 파운데이션 모델을 만드는데 약 7억장의 문서가 필요한 셈(??)
- 특정 도메인에 맞춰서 새롭게 파인튜닝을 할 때 기존 학습데이터의 0.01%만큼만 쓴다고 하더라도 7만장이 필요하게 된다(?)
- `RAG는 유저가 사용하는 프롬프트에 추가하고 싶은 데이터를 모델에 관계없이 적용시킬 수 있어 값비싼 파인 튜닝 없이 LLM을 보다 효율적으로 사용`
  
- RAG를 사용하면 기업은 데이터 관련성을 유지하고 비용을 최적화하면서 사용자 지정 솔루션을 달성할 수 있다.
- RAG는 미세 조정 없이 정기적인 데이터 업데이트를 촉진하여 LLM을 비즈니스에 효율적으로 통합

##### RAG 단계
1. 문서를 청크 단위로 세분화
2. 세분화한 문서 청크에서 자연어 임베딩 생성후 DB에 인덱싱
3. 사용자의 쿼리(질문)에 대한에 대한 벡터 임베딩 생성.
4. `벡터 유사도 검색`을 사용자 쿼리와 자연어 임베딩 DB에서 관련 문서 검색 유사도 검색을 사용하여 `유사하다고 생각되는 top K개의 문서 청크를 프롬프트에 연결`


#### 임베딩 과 벡터
- RAG에서 사용되는 현재 최신 기법중 하나는 주어진 사용자 질의에 대한 의미론적 유사성을 계산하기 위해 지식 베이스의 조밀한 벡터 표현을 만드는 것
- 예시로는 SentenceTransformers와 `all-MiniLM-L6-v2`를 사용할 예정
- 벡터 데이터베이스란 기존에 저희가 사용하던 DB와 비슷하나 벡터 인덱싱과 색인에 강점이 있다.

-----

#### Milvus 란?
- Milvus 는 심층 신경망 및 기타 기계 학습(ML) 모델에서 생성된 대규모 임베딩 벡터를 저장, 색인 및 관리하는 단일 목표로 하는 벡터 데이터베이스(Vector Database) 입니다.
- https://jackerlab.com/milvus/#:~:text=Milvus%20%EB%9E%80%3F,%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B2%A0%EC%9D%B4%EC%8A%A4(Vector%20Database)%20%EC%9E%85%EB%8B%88%EB%8B%A4.

#### Milvus를 활용한 임베딩 데이터 추출 및 저장
- Milvus는 VectorDB로 주어진 문서를 자연어 임베딩을 기반으로 쿼리를 실행할 수 있도록 만들어 졌습니다.
- 이번 시간에는 자연어 모델인 BERT 모델의 소형 버전을 활용하여 시멘틱 서치 수행

##### 1. Milvus DB connection
- pymilvus를 사용하거나
- LangChain의 Milvus 벡터스토어 클래스를 사용하여 문서 인스턴스를 추가(Milvus DB connection)

##### 2. Milvus store에 DB schema 선언
- collection 이름이 같지만 다른 schema가 선언된 경우 SchemaNotReady exception이 발생할 수 있으니, collection과 schema는 pair로 관리되어야(collection과 schema는 1:1 페어)
- 텍스트 필드의 최대 길이는 문자가 아닌 `바이트 단위`로 계산됩니다. 문자열의 바이트 크기를 가져와 스키마의 바이트 제한에 맞게 잘라내어 DB성능의 향상을 꾀할 수도 있지만, `DB내 생성될 record들의 데이터 길이에 대한 사전 정보가 없기 때문에` 허용되는 최대값(65535)으로 제한을 설정하는 것이 좋습니다.
- 특정 필드는 사전에 정의된 namespace를 지켜 주어야 한다.
    - primary key는 반드시 "pk"로 선언되어야 한다.
    - 계산된 vector는 "vector"로 선언되어야 한다.
    - text entry는 반드시 "text"로 선언되어야 한다.

In [ ]:
# Milvus requires a connection for all operations. Remember to disconnect at the end.

from pymilvus import connections
from pymilvus import CollectionSchema, FieldSchema, DataType, Collection, utility
connections.connect(
  alias="default",
  host= "127.0.0.1", #"127.0.0.1", # YOUR IP
  port= "9090" #"19530"      # YOUR PORT
)

# schemaless 동작을 원하면 if `enable_dynamic_fields=True`.
MAX_TITLE = 512
MAX_TEXT = 1024
MAX_VEC = 384

NAME = "WikiHow"

if NAME in utility.list_collections(): # utility? 
    whcollection = Collection(NAME)    # Collection? 
    whcollection.drop() # 이미 utility.list_collections()에 해당 이름이 있다면 드롭한다.

whschema = CollectionSchema( # CollectionSchema?
  fields=[                   # fileds? FieldSchema?
    FieldSchema(name="pk",  dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=65535, default_value="")
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65535, default_value="")
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=384)
  ],
  enable_dynamic_fields=False
  description="WikiHow collection"
)

whcollection=Collection( # Collection?
  name=NAME,
  schema=whschema,
  consistency_level="Session" # 우리가 쓴 글을 반드시 읽어야 합니다. 그렇지 않으면 약간 오래된 내용이 될 수 있습니다.
)

#### 3. MilvusDB 사용시 Batch 기반 입력
- 작은 BERT 모델을 사용하여 Milvus store내 배치할 문서에 대한 임베딩을 계산할 예정
- 

-----

#### Q&A
Q. CromaDB 랑 Mavilas 차이?
- 